Step 1 - Install Tesseract:
 % brew install Tesseract

Step 2 - Install pytesseract
Python-tesseract is an optical character recognition (OCR) tool for python. 
 % pip install pytesseract 

Step 1 - You can varify whether tesseract is properly installed or not by using version command
 % tesseract --version
tesseract 5.3.2

Step 3 - To check where the tesseract is installed, use this command. 
 % where tesseract    
/usr/local/bin/tesseract

keep this location safe. You can find file named "tesseract" in this folder. Right click on it and save its location for future reference.

Step 3 - Install numpy
 % pip3 install numpy

Step 3 - Install numpy
 % pip3 install openai
 
Step - Install open cv
 % pip3 install opencv-python
 Successfully installed opencv-python-4.8.0.76
 
 # Trying another OCR method
Step - Install easyocr
% pip install easyocr   


In [21]:
from PIL import Image
import pytesseract
import numpy as np
import openai
import os
import cv2
import easyocr

In [43]:
# path to tesseract file
pytesseract.pytesseract.tesseract_cmd = r'/usr/local/Cellar/tesseract/5.3.2_1/bin/tesseract'
# Please add your scanned file image below
filePath = '/Users/mac/Downloads/testimage.png'

scanned_img = Image.open(filePath)
scanned_text = pytesseract.image_to_string(scanned_img)
# Batch processing with a single file containing the list of multiple image file paths
# print(pytesseract.image_to_string('images.txt'))


# # for testing purpose
# reader = easyocr.Reader(['en'])
# result = reader.readtext(filePath, detail=0)
# print("EasyOCR Result: \n",result)

if len(scanned_text) == 0:
    print("Unable to read text from image")
else:
    print("Tesseract Result: \n",scanned_text)

Tesseract Result: 
 (b) What are the circumstances where revenue expenditure
becomes capital expenditure? 5

Ans: Revenue expenditure is treated as capital expenditure in the
following cases:

1. Raw material consumed during a period: Raw materials
consumed in the production of goods are treated as revenue
expenditure but if such raw materials are consumed in the
construction of fixed assets are treated as capital expenditure.

2. Wages: Wages paid to labourers which are engaged in the
production of goods are treated as revenue expenditure but if
wages paid to workers who are engaged in the construction of
fixed assets, then such wages are treated as capital
expenditure.

3. Repairs: Repairing expenses are of nominal nature and debited
to profit and loss account. But repairing expenses incurred on
old or second hand assets which increase the capacity or life of
the assets are treated as revenue expenditure.

4. Legal charges: Legal charges are normally debited to profit and
loss accoun

In [5]:
prompt = f"""Your task is to determine if the student's solution \
is correct or not. \

If its more than 50% correct then classify it as correct. \

To solve the problem do the following:\

- First, work out your own solution to the problem.\

- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.\

Don't decide if the student's solution is correct until you have done the problem yourself.\

Use the following format:\

Question:\n
---\n
put the question here\n
---\n
Student's solution:\n
---\n
Write Student's solution here\n
---\n
Actual solution:\n
---\n
write steps to work out the solution and your solution here. If there are no steps then you can just write your solution here \n
---\n
Is the student's solution the same as actual solution you just calculated? \n
---\n
yes or no\n
---\n
Student grade:\n
---\n
correct or incorrect\n
---\n

Solution: ```{scanned_text}```
"""


# print(prompt)

In [2]:
# please add your api key below
openai.api_key = "<Plaese add your api key here>"

In [3]:

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.9,
    )
    return response.choices[0].message["content"]


In [ ]:
response = get_completion(prompt)
print(response)

# Resources:
https://pypi.org/project/pytesseract/

# Other useful commands

# Batch processing with a single file containing the list of multiple image file paths
print(pytesseract.image_to_string('images.txt'))

# Timeout/terminate the tesseract job after a period of time
try:
    print(pytesseract.image_to_string('test.jpg', timeout=2)) # Timeout after 2 seconds
    print(pytesseract.image_to_string('test.jpg', timeout=0.5)) # Timeout after half a second
except RuntimeError as timeout_error:
    # Tesseract processing is terminated
    pass
    
    


# Resource:
https://nanonets.com/blog/ocr-with-tesseract/

# Preprocessing for Tesseract
To avoid all the ways your tesseract output accuracy can drop, you need to make sure the image is appropriately pre-processed.

This includes rescaling, binarization, noise removal, deskewing, etc.

To preprocess image for OCR, use any of the following python functions.

import cv2
import numpy as np

img = cv2.imread('image.jpg')

# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
# thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
# erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

# opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

# skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
     if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
 
 
# examples
    
image = cv2.imread('aurebesh.jpg')
gray = get_grayscale(image)
thresh = thresholding(gray)
opening = opening(gray)
canny = canny(gray)

# Limitations:
It is not capable of recognizing handwriting.


https://pspdfkit.com/blog/2023/how-to-use-tesseract-ocr-in-python/
